Commets and Asteroids

Asteroids 

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astroquery.jplhorizons import Horizons


# list object
object_plannet = ['SUN', '199', '299', '399', '301', '-74', '599', '699', '433', 'Eros', 'Orus', '-49',
                       '-490', '-255', '2000016', '90000190',  
                        'Leucus', 'Eurybates','Polymele', 'Vesta', 'Mathilde', 'Lutetia', 'Donaldjohanson','Braille', 'Annefrank', 
                        'Bennu', 'Itokawa', '-64', 'Apophis', 'Ryugu', '-122911', 'Patroclus', 
                        '90000855', 'Dinkinesh', '20065803' , 'Gaspra', 'Ceres']
"""
+------------------------------------------------------------------------+
ID          :   Asteroid name
1               Mercury Barycenter                                                   
199             Mercury 
2               Venus Barycenter                                                     
299             Venus                                                                
-248            Venus Express (spacecraft)
3               Earth Moon Barry Center
399             Earth
-74             Mars Reconnaissance Orbiter (spacec                   
-76             Mars Science Laboratory (spacecraft   -> Errors              
-143            ExoMars16 TGO (spacecraft)                                           
-168            Mars2020 (spacecraft) 
5               Jupiter Barycenter                                                   
599             Jupiter 
6               Saturn Barycenter                                                    
699             Saturn 
+------------------------------------------------------------------------+
-49             Lucy Centaur RB Booster (spacecraft
433             Eros
-490            Lucy
-255            Psyche
-64             OSIRIS-REx
-64090          OSIRIS-REx  SRC     # cannot found at now
90000188        Tempel 1
90000189        Tempel 1
90000190        Tempel 1
90000191        Tempel 1
709             Cressida        not use                
-122911         Solidaridad-1 (spacecraft)
20065803        Didymos (system barycenter)                                          
920065803       Didymos (primary body)                                               
120065803       Dimorphos
"""
"""
    Objects                      ID code #
    ---------------------------  -------------------
    All planet barycenters       0,1,2,3,4,5,6,7,8,9
    Sun                          10
    Moon                         301
    Mercury                      199
    Venus                        299
    Earth                        399
"""
# yyyy-mm-dd hh:mm:ss
epoch = Time('2024-01-13 18:00:00')

# object_name = 'Orus'
q = Horizons(object_plannet[-1], location='@0', epochs=epoch.tdb.jd)
tab = q.vectors(refplane='earth')
c = SkyCoord(tab['x'].quantity, tab['y'].quantity, tab['z'].quantity,
             representation_type='cartesian', frame='icrs',
             obstime=epoch)

c

: 

In [3]:
epoch

c:\Users\pang0\AppData\Local\Programs\Python\Python310\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


<Time object: scale='utc' format='iso' value=1950-01-13 18:00:00.000>

Calculate Position 

In [ ]:
import re

def calculate_position(obj, time):
    """
    Args : 
        obj     : obj name str
        time    : time (utc) format yyyy-mm-dd
    """
    epoch = Time(time)
    q = Horizons(obj, location='@0', epochs=epoch.tdb.jd)
    tab = q.vectors(refplane='earth')
    c = SkyCoord(tab['x'].quantity, tab['y'].quantity, tab['z'].quantity,
                representation_type='cartesian', frame='icrs',
                obstime=epoch)
    result = str(c)
    matches = re.findall(r'-?\d+\.\d+', result)
    result_list = [float(match) for match in matches]
    return result_list

ax = calculate_position(object_plannet[0] ,'2000-01-01')
ax

: 

In [5]:

start_date = '1970-01-01'
end_date = '2024-01-14'

date_range = pd.date_range(start=start_date, end=end_date, freq='D')

df = pd.DataFrame({'Date': date_range})
df.to_csv('time.csv')

In [ ]:
import pandas as pd

time = pd.read_csv('time.csv')
time['time'] = pd.to_datetime(time['Date'])
time = time[time['time'] >= '2010-01-01'].astype(str)
time = time['time']


In [ ]:
from threading import Thread

rl = []

class Main(Thread):
    def __init__(self, i):
        super().__init__()
        self.i = i
    def run(self):
        for t in time:
            try:
                ls = calculate_position(object_plannet[self.i], t)
                rl.append(ls)
            except Exception as e:
                print(f"Error calculating position for object {i} at time {t}: {e}")
                rl.append([99, 99, 99])

for i in range(len(object_plannet)):
    Main(i).start()
    if   len(rl)%3000 == 0 or len(rl) == len(time):
        results = rl
        columns = ['Time'] + [f'{obj}_{coord}' for obj in object_plannet for coord in ['X', 'Y', 'Z']]
        results_df = pd.DataFrame(results, columns=columns)
        results_df.to_csv(f'csv{len(rl)}.csv')

